In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('ocs.csv')

In [3]:
df.dtypes

name              object
price             object
price_per_gram    object
grams             object
type              object
producer          object
brand             object
potency           object
thc               object
cbd               object
dtype: object

In [4]:
old_shape = df.shape

In [5]:
df.drop_duplicates(inplace=True)
new_shape = df.shape

In [6]:
old_shape[0] - new_shape[0] #number of duplicates

51

In [7]:
# Remove '$' from price_per_gram column
df['price_per_gram'] = df['price_per_gram'].map(lambda x: x.replace('$',''))

In [8]:
df['thc'] = df['thc'].apply(lambda x: x.replace('%',''))
df['cbd'] = df['cbd'].apply(lambda x: x.replace('%',''))

In [9]:
# df.apply(lambda x: x.str.split(', ')).explode(['price', 'price_per_gram', 'grams'])
# .reset_index().reindex(df.columns, axis=1)
df.price = df.price.astype('string').apply(lambda x: x.split(', '))
# df.price_per_gram = df.price_per_gram.astype('string').apply(lambda x: x.replace(' $', ''))
df.price_per_gram = df.price_per_gram.apply(lambda x: re.sub(' $', '', x))
df.price_per_gram = df.price_per_gram.apply(lambda x: re.sub(' ,', ',', x))
df.price_per_gram = df.price_per_gram.astype('string').apply(lambda x: x.split(', '))
df.grams = df.grams.astype('string').apply(lambda x: x.split(', '))

In [10]:
#split thc and cbd range
df['thc_min'] = df['thc'].apply(lambda x: x.split(' - ')[0]).astype('float')
df['thc_max'] = df['thc'].apply(lambda x: x.split(' - ')[1]).astype('float')
df['cbd_min'] = df['cbd'].apply(lambda x: x.split(' - ')[0]).astype('float')
df['cbd_max'] = df['cbd'].apply(lambda x: x.split(' - ')[-1].replace('  ', '')).astype('float')
df = df.drop(columns = ['thc', 'cbd'])

Find rows with unequal number of elements

In [11]:
df = df.sort_index()
index = -1
false_indices = []
for i, j in zip(df['price'], df['price_per_gram']):
    index += 1
    if len(i) != len(j):
        false_indices.append(index)
false_indices

[71]

In [12]:
[df.iloc[71, 1][0]] + df.iloc[71, 2]
df.at[71, 'price_per_gram'] = [df.iloc[71, 1][0]] + df.iloc[71, 2]

In [13]:
# Set columns to be untouched as indices
df = df.set_index(['name', 'type', 'producer', 'brand', 'potency', 'thc_min', 'thc_max', 'cbd_min', 'cbd_max'])
df = df[~df.index.duplicated(keep='first')]
df.index.is_unique

True

In [14]:
df = df.explode(column = ['price', 'price_per_gram', 'grams'])
df = df.reset_index()

In [15]:
df.loc[df.potency == "Very Strong", "potency"] = "5"
df.loc[df.potency == "Strong", "potency"] = "4"
df.loc[df.potency == "Medium", "potency"] = "3"
df.loc[df.potency == "Mild", "potency"] = "2"
df.loc[df.potency == "Little to Minimal", "potency"] = "1"
df.potency.value_counts()

5    400
4     32
2     10
3      9
1      5
Name: potency, dtype: int64

In [16]:
df.to_csv('ocs_cleaned.csv', index=False)